![Logo BV IBMEC](https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/logo-bv-ibmec-notebooks.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ian-iania/IBMEC-BV-Modelos-Preditivos/blob/main/notebooks/NB04_Clustering.ipynb)

# NB04 — Clustering (K-Means) e Segmentação de Carteira (Auto com LTV)

- carteira média engana: segmentar por semelhança
- K-Means cria clusters; PCA ajuda a visualizar
- tabela por cluster vira plano FP&A (mix decide)

## 1) Contexto do bloco

### O que vamos fazer
Usar **clustering não supervisionado** para separar perfis de carteira Auto.

### Por que isso importa em FP&A
A média da carteira esconde comportamento importante de risco, ticket, canal e prazo.

### O que observar no output
No final teremos três entregáveis de negócio: scatter PCA, tabela executiva por cluster e plano de ação por segmento.

### 1.1) Imports da aula

### O que vamos fazer
Carregar bibliotecas para dados, clustering, PCA e visualização.

### Por que isso importa em FP&A
São as ferramentas mínimas para transformar dados em segmentação acionável.

### O que observar no output
A célula roda sem output complexo; é apenas preparação do ambiente.

In [ ]:
import pandas as pd  # biblioteca principal para manipulação de dados tabulares
import numpy as np  # biblioteca para operações numéricas
import matplotlib.pyplot as plt  # biblioteca para histogramas simples

from sklearn.preprocessing import StandardScaler  # padronização das variáveis numéricas
from sklearn.cluster import KMeans  # algoritmo principal de clustering
from sklearn.decomposition import PCA  # redução de dimensão para visualização 2D
from sklearn.metrics import silhouette_score  # métrica opcional de qualidade de cluster

import plotly.express as px  # gráficos interativos rápidos
import plotly.graph_objects as go  # gráficos interativos customizados

## 2) Carregar e entender o dataset (sanidade mínima)

### O que vamos fazer
Ler a base sintética de clustering direto do GitHub raw e ordenar no tempo.

### Por que isso importa em FP&A
Fluxo reproduzível no Colab evita divergência entre os alunos durante a aula.

### O que observar no output
Verifique tamanho da base e primeiras linhas com a coluna `dt` já convertida para data.

In [ ]:
url = "https://raw.githubusercontent.com/ian-iania/IBMEC-BV-Modelos-Preditivos/main/data/bv_auto_clustering_sintetico.csv"  # URL oficial da base

df = pd.read_csv(url, parse_dates=["dt"])  # lê CSV e converte a coluna de data
df = df.sort_values("dt").reset_index(drop=True)  # ordena cronologicamente para consistência

### 2.1) Primeira leitura da base

### O que vamos fazer
Exibir shape e `head(3)` para confirmar estrutura.

### Por que isso importa em FP&A
Checagem simples evita continuar a aula com base errada ou colunas faltantes.

### O que observar no output
Quantidade de linhas/colunas e se as colunas de negócio estão presentes.

In [ ]:
print(f"Shape: {df.shape[0]} linhas x {df.shape[1]} colunas")  # imprime tamanho da base

df.head(3)  # mostra as primeiras linhas para inspeção inicial

### 2.2) Sanidade rápida de variáveis-chave

### O que vamos fazer
Ver distribuição de `canal` e estatísticas descritivas das variáveis mais importantes.

### Por que isso importa em FP&A
Antes do clustering, precisamos garantir que os drivers têm comportamento plausível.

### O que observar no output
Faixas de LTV, PTI, score e PD proxy com ordem de grandeza coerente.

In [ ]:
print("Distribuição de canal:")  # título para leitura rápida
print(df["canal"].value_counts(dropna=False))  # mostra frequência dos canais

cols_stats = ["ltv", "loan_amount", "pti", "score_interno", "pd_proxy"]  # seleciona colunas de resumo

df[cols_stats].describe().T  # exibe estatísticas por variável

### 2.3) Histograma de LTV

### O que vamos fazer
Visualizar distribuição de LTV da carteira.

### Por que isso importa em FP&A
LTV é um dos principais drivers para entender perfil de risco e política de crédito.

### O que observar no output
Concentração da carteira em faixas de LTV e possíveis caudas altas.

In [ ]:
fig_ltv = px.histogram(  # cria histograma interativo de LTV
    df,
    x="ltv",
    nbins=40,
    title="PARTE A — Distribuição de LTV"
)
fig_ltv.update_layout(xaxis_title="LTV", yaxis_title="Quantidade")  # define títulos dos eixos
fig_ltv.show()  # renderiza o gráfico

**Leitura FP&A:** esse histograma ajuda a antecipar grupos de maior sensibilidade a risco e pricing.

### 2.4) Histograma de PD proxy

### O que vamos fazer
Visualizar a distribuição de risco estimado (`pd_proxy`).

### Por que isso importa em FP&A
A cauda de risco influencia muito o desenho de segmentos e ações por cluster.

### O que observar no output
Veja se há massa concentrada em baixo risco e cauda para risco mais alto.

In [ ]:
fig_pd = px.histogram(  # cria histograma interativo de PD proxy
    df,
    x="pd_proxy",
    nbins=40,
    title="PARTE A — Distribuição de PD proxy"
)
fig_pd.update_layout(xaxis_title="PD proxy", yaxis_title="Quantidade")  # ajusta nomes dos eixos
fig_pd.show()  # exibe gráfico

**Leitura FP&A:** a cauda de `pd_proxy` é útil para distinguir clusters mais defensivos vs mais agressivos.

### 2.5) Barras por canal (visão rápida)

### O que vamos fazer
Comparar volume por canal de origem.

### Por que isso importa em FP&A
Mix de canal costuma aparecer como diferencial entre clusters.

### O que observar no output
Participação relativa de `digital`, `agencia` e `parceiro`.

In [ ]:
canal_counts = df["canal"].value_counts().reset_index()  # calcula contagem por canal
canal_counts.columns = ["canal", "qtd"]  # renomeia colunas para clareza

fig_canal = px.bar(  # cria gráfico de barras por canal
    canal_counts,
    x="canal",
    y="qtd",
    title="PARTE A — Volume por canal"
)
fig_canal.update_layout(xaxis_title="Canal", yaxis_title="Quantidade")  # ajusta eixos
fig_canal.show()  # exibe gráfico

**Leitura FP&A:** este mix será revisitado na tabela executiva por cluster para orientar ações comerciais.

## 3) Selecionar features para clustering

### O que vamos fazer
Definir as variáveis que entram no K-Means e preparar matriz `X`.

### Por que isso importa em FP&A
K-Means usa distância; escolher features certas evita segmentação enviesada.

### O que observar no output
`X` final deve incluir numéricas + one-hot de canal, sem usar `pd_proxy`, `risk_band` e `cluster_true` no treinamento.

In [ ]:
num_features = [  # lista de variáveis numéricas para clustering
    "ltv", "loan_amount", "term_meses", "taxa_mensal", "pti", "score_interno",
    "utilizacao_limite", "atraso_antes_30d", "atraso_antes_60d", "idade", "renda_mensal"
]

X_num = df[num_features].copy()  # recorta parte numérica da matriz de features
X_cat = pd.get_dummies(df[["canal"]], prefix="canal", dtype=float)  # faz one-hot de canal
X = pd.concat([X_num, X_cat], axis=1)  # concatena numéricas com dummies de canal

### 3.1) Conferir matriz final de features

### O que vamos fazer
Exibir shape e nomes das colunas de `X`.

### Por que isso importa em FP&A
Transparência do input evita erro de interpretação no resultado dos clusters.

### O que observar no output
Confirme ausência de `pd_proxy`, `risk_band` e `cluster_true` em `X`.

In [ ]:
print(f"Shape de X: {X.shape[0]} linhas x {X.shape[1]} colunas")  # imprime dimensões da matriz de features
print("Colunas de X:")  # título visual da lista
print(list(X.columns))  # mostra lista de colunas finais do clustering

## 4) Padronização (StandardScaler)

### O que vamos fazer
Padronizar as features para média 0 e desvio 1.

### Por que isso importa em FP&A
Sem padronizar, variáveis com escala alta (ex.: ticket) dominam a distância e distorcem clusters.

### O que observar no output
Dimensão de `X_scaled` igual a `X` e estatísticas próximas de média 0 / desvio 1.

In [ ]:
scaler = StandardScaler()  # instancia padronizador
X_scaled = scaler.fit_transform(X)  # ajusta e transforma a matriz de features

print(f"Shape de X_scaled: {X_scaled.shape}")  # confirma dimensões após padronização
print(f"Média aproximada da 1ª coluna: {X_scaled[:, 0].mean():.3f}")  # checagem rápida de centralização
print(f"Desvio aproximado da 1ª coluna: {X_scaled[:, 0].std():.3f}")  # checagem rápida de escala

## 5) K-Means (K=4)

### O que vamos fazer
Rodar K-Means com 4 clusters e adicionar resultado no DataFrame.

### Por que isso importa em FP&A
Quatro clusters costumam ser um número acionável para discutir estratégia de mix.

### O que observar no output
Volume por cluster e equilíbrio relativo dos grupos.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)  # instancia K-Means com K=4 e seed fixa

df["cluster_kmeans"] = kmeans.fit_predict(X_scaled)  # ajusta modelo e grava cluster em cada contrato

cluster_counts = df["cluster_kmeans"].value_counts().sort_index().reset_index()  # conta contratos por cluster
cluster_counts.columns = ["cluster_kmeans", "qtd"]  # renomeia colunas para leitura
cluster_counts  # exibe contagem por cluster

### 5.1) Silhouette (checagem opcional rápida)

### O que vamos fazer
Calcular uma métrica simples de separação dos clusters.

### Por que isso importa em FP&A
É uma checagem técnica rápida para evitar segmentação sem sinal de separação.

### O que observar no output
Quanto mais perto de 1, melhor separação; perto de 0 indica sobreposição forte.

In [ ]:
sil_k4 = silhouette_score(X_scaled, df["cluster_kmeans"])  # calcula silhouette para K=4
print(f"Silhouette (K=4): {sil_k4:.3f}")  # imprime métrica em formato curto

## 6) PCA 2D para visual executivo

### O que vamos fazer
Projetar as features padronizadas em dois componentes principais (`pc1` e `pc2`).

### Por que isso importa em FP&A
PCA não cria cluster, mas facilita comunicar a segmentação em um gráfico 2D.

### O que observar no output
Percentual de variância explicado por PC1+PC2 e separação visual entre cores.

In [ ]:
pca = PCA(n_components=2, random_state=42)  # instancia PCA com dois componentes
pcs = pca.fit_transform(X_scaled)  # ajusta PCA e transforma X_scaled em duas dimensões

df["pc1"] = pcs[:, 0]  # grava primeiro componente no DataFrame
df["pc2"] = pcs[:, 1]  # grava segundo componente no DataFrame

explained = pca.explained_variance_ratio_.sum()  # soma variância explicada por PC1+PC2
print(f"Variância explicada por PC1+PC2: {explained:.2%}")  # imprime resumo de explicação

### 6.1) Scatter de PCA colorido por cluster (Plotly)

### O que vamos fazer
Plotar `pc1` x `pc2` com cor por cluster do K-Means.

### Por que isso importa em FP&A
É a visualização central para explicar segmentação para público não técnico.

### O que observar no output
Se as nuvens aparecem separadas, a segmentação está mais clara para comunicação executiva.

In [ ]:
fig_pca = px.scatter(  # cria scatter 2D da projeção PCA
    df,
    x="pc1",
    y="pc2",
    color=df["cluster_kmeans"].astype(str),
    hover_data=["ltv", "loan_amount", "term_meses", "canal", "pti", "score_interno", "pd_proxy"],
    title="PARTE B — PCA 2D com clusters do K-Means"
)
fig_pca.update_layout(xaxis_title="PC1", yaxis_title="PC2", legend_title_text="Cluster")  # configura eixos e legenda
fig_pca.show()  # exibe gráfico interativo

**Leitura FP&A:** nuvens mais separadas sugerem perfis de carteira mais distintos e mais fáceis de transformar em estratégia.

## 7) Tabela executiva por cluster (entregável FP&A)

### O que vamos fazer
Gerar um resumo por cluster com tamanho, risco, ticket, prazo, LTV, PTI, score, atrasos e canal dominante.

### Por que isso importa em FP&A
Essa tabela é o insumo direto para decisão de mix e definição de ações por segmento.

### O que observar no output
Comparar rapidamente quais clusters concentram risco maior, ticket maior e perfil de canal diferente.

In [ ]:
cluster_summary = df.groupby("cluster_kmeans").agg(  # agrega métricas principais por cluster
    qtd=("id", "count"),
    pd_proxy_mean=("pd_proxy", "mean"),
    pd_proxy_median=("pd_proxy", "median"),
    loan_amount_mean=("loan_amount", "mean"),
    term_meses_mean=("term_meses", "mean"),
    ltv_mean=("ltv", "mean"),
    pti_mean=("pti", "mean"),
    score_interno_mean=("score_interno", "mean"),
    atraso30_rate=("atraso_antes_30d", "mean"),
    atraso60_rate=("atraso_antes_60d", "mean")
).reset_index()

cluster_summary["pct_carteira"] = cluster_summary["qtd"] / len(df)  # calcula participação de cada cluster na carteira

### 7.1) Canal dominante de cada cluster

### O que vamos fazer
Calcular o canal com maior frequência em cada grupo.

### Por que isso importa em FP&A
Canal dominante ajuda a direcionar plano comercial e governança de aquisição.

### O que observar no output
Cada cluster terá um canal dominante associado.

In [ ]:
canal_mode = df.groupby(["cluster_kmeans", "canal"]).size().reset_index(name="qtd_canal")  # conta ocorrências por cluster e canal
canal_mode = canal_mode.sort_values(["cluster_kmeans", "qtd_canal"], ascending=[True, False])  # ordena para identificar o dominante
canal_mode = canal_mode.drop_duplicates("cluster_kmeans")[ ["cluster_kmeans", "canal"] ]  # mantém apenas canal dominante por cluster
canal_mode = canal_mode.rename(columns={"canal": "canal_dominante"})  # renomeia coluna para leitura executiva

cluster_summary = cluster_summary.merge(canal_mode, on="cluster_kmeans", how="left")  # incorpora canal dominante na tabela resumo

### 7.2) Formatar tabela para leitura de slide

### O que vamos fazer
Montar uma versão legível com percentuais e valores monetários.

### Por que isso importa em FP&A
Formato executivo acelera discussão em sala e em comitê.

### O que observar no output
Tabela curta e pronta para interpretação imediata.

In [ ]:
cluster_display = cluster_summary.copy()  # cria cópia para formatar sem perder base numérica

cluster_display["% carteira"] = (cluster_display["pct_carteira"] * 100).round(1).astype(str) + "%"  # formata participação
cluster_display["pd_proxy médio"] = (cluster_display["pd_proxy_mean"] * 100).round(2).astype(str) + "%"  # formata risco médio
cluster_display["pd_proxy mediana"] = (cluster_display["pd_proxy_median"] * 100).round(2).astype(str) + "%"  # formata risco mediano
cluster_display["loan médio (R$)"] = cluster_display["loan_amount_mean"].round(0).map(lambda x: f"R$ {x:,.0f}")  # formata ticket
cluster_display["atraso30 (%)"] = (cluster_display["atraso30_rate"] * 100).round(1).astype(str) + "%"  # formata atraso 30d
cluster_display["atraso60 (%)"] = (cluster_display["atraso60_rate"] * 100).round(1).astype(str) + "%"  # formata atraso 60d

cluster_display[["cluster_kmeans", "% carteira", "pd_proxy médio", "pd_proxy mediana", "loan médio (R$)", "term_meses_mean", "ltv_mean", "pti_mean", "score_interno_mean", "canal_dominante", "atraso30 (%)", "atraso60 (%)"]]

**Leitura FP&A:** esta é a tabela principal do bloco. É ela que transforma segmentação em decisão de mix.

### 7.3) Gráfico executivo: % da carteira por cluster

### O que vamos fazer
Comparar tamanho relativo dos clusters.

### Por que isso importa em FP&A
Mesmo cluster de risco alto pode ter impacto moderado se o peso na carteira for pequeno.

### O que observar no output
Participação de cada cluster no total da carteira.

In [ ]:
fig_mix = px.bar(  # cria barras de participação por cluster
    cluster_summary,
    x="cluster_kmeans",
    y="pct_carteira",
    title="PARTE C — % da carteira por cluster",
    text=cluster_summary["pct_carteira"].map(lambda x: f"{x:.1%}")
)
fig_mix.update_layout(xaxis_title="Cluster", yaxis_title="% da carteira")  # ajusta eixos
fig_mix.show()  # exibe gráfico

### 7.4) Gráfico executivo: PD proxy médio por cluster

### O que vamos fazer
Comparar risco médio entre os grupos.

### Por que isso importa em FP&A
Prioriza onde calibrar política, pricing e monitoramento.

### O que observar no output
Clusters com maior `pd_proxy_mean` merecem atenção de risco.

In [ ]:
fig_pd_cluster = px.bar(  # cria barras de PD proxy médio por cluster
    cluster_summary,
    x="cluster_kmeans",
    y="pd_proxy_mean",
    title="PARTE C — PD proxy médio por cluster",
    text=cluster_summary["pd_proxy_mean"].map(lambda x: f"{x:.2%}")
)
fig_pd_cluster.update_layout(xaxis_title="Cluster", yaxis_title="PD proxy médio")  # ajusta eixos
fig_pd_cluster.show()  # renderiza gráfico

### 7.5) Gráfico executivo: loan_amount médio por cluster

### O que vamos fazer
Comparar ticket médio de crédito entre clusters.

### Por que isso importa em FP&A
Ticket ajuda a medir relevância financeira de cada segmento.

### O que observar no output
Clusters de maior ticket podem exigir gestão diferenciada de risco-retorno.

In [ ]:
fig_ticket = px.bar(  # cria barras de loan médio por cluster
    cluster_summary,
    x="cluster_kmeans",
    y="loan_amount_mean",
    title="PARTE C — Loan amount médio por cluster",
    text=cluster_summary["loan_amount_mean"].round(0).map(lambda x: f"R$ {x:,.0f}")
)
fig_ticket.update_layout(xaxis_title="Cluster", yaxis_title="Loan amount médio (R$)")  # configura eixos
fig_ticket.show()  # exibe gráfico

## 8) Nomear clusters em linguagem de negócio

### O que vamos fazer
Converter IDs técnicos de cluster em nomes acionáveis de negócio.

### Por que isso importa em FP&A
Cluster sem nome não vira plano nem dono de ação.

### O que observar no output
Cada cluster terá nome e recomendação prática de 1 linha.

In [ ]:
cluster_summary = cluster_summary.sort_values("cluster_kmeans").reset_index(drop=True)  # ordena clusters para padronizar leitura
cluster_summary["rank_risco"] = cluster_summary["pd_proxy_mean"].rank(ascending=False, method="dense")  # ranqueia risco médio
cluster_summary["rank_ticket"] = cluster_summary["loan_amount_mean"].rank(ascending=False, method="dense")  # ranqueia ticket médio

nomes = []  # lista para nomes sugeridos
acoes = []  # lista para ações sugeridas

for _, row in cluster_summary.iterrows():  # percorre clusters para definir nome e ação
    if row["rank_risco"] == 1:
        nomes.append("Risco Alto / Vigilância")
        acoes.append("Ajustar pricing e restringir exposição; reforçar monitoramento mensal.")
    elif row["rank_risco"] == 4:
        nomes.append("Prime / Crescimento")
        acoes.append("Manter política e crescer com foco em canal dominante e eficiência comercial.")
    elif row["rank_ticket"] == 1:
        nomes.append("Ticket Alto / Atenção")
        acoes.append("Controlar concentração e calibrar limites para proteger retorno ajustado ao risco.")
    else:
        nomes.append("Intermediário / Balanceado")
        acoes.append("Manter mix balanceado com ajustes pontuais de política e acompanhamento de atraso.")

cluster_summary["nome_cluster"] = nomes  # adiciona nome executivo por cluster
cluster_summary["acao_recomendada"] = acoes  # adiciona ação recomendada por cluster

### 8.1) Tabela final: cluster, nome e plano FP&A

### O que vamos fazer
Exibir um quadro final pronto para discussão de sala e comitê.

### Por que isso importa em FP&A
Esse quadro já conecta segmentação com plano de ação e governança.

### O que observar no output
Nome coerente, 2 métricas-chave e ação clara para cada cluster.

In [ ]:
plan_table = cluster_summary[[  # seleciona colunas finais do plano
    "cluster_kmeans", "nome_cluster", "pd_proxy_mean", "ltv_mean", "canal_dominante", "acao_recomendada"
]].copy()

plan_table["pd_proxy médio"] = (plan_table["pd_proxy_mean"] * 100).round(2).astype(str) + "%"  # formata risco médio
plan_table["ltv médio"] = plan_table["ltv_mean"].round(3)  # formata LTV médio

plan_table[["cluster_kmeans", "nome_cluster", "pd_proxy médio", "ltv médio", "canal_dominante", "acao_recomendada"]]

**Leitura FP&A:** aqui termina a parte técnica e começa a parte de gestão: cada cluster agora tem narrativa e ação objetiva.

## 9) Validação rápida do professor (opcional)

### O que vamos fazer
Comparar `cluster_true` (somente didático) com `cluster_kmeans`.

### Por que isso importa em FP&A
Serve como referência de calibração para a aula, mas em produção não existe rótulo verdadeiro.

### O que observar no output
Concentração de massa na diagonal indica alinhamento maior entre clusters sintéticos e K-Means.

In [ ]:
crosstab_clusters = pd.crosstab(df["cluster_true"], df["cluster_kmeans"], margins=True)  # cruza cluster sintético com cluster estimado
crosstab_clusters  # exibe tabela de validação opcional

**Leitura FP&A:** use esta tabela apenas como termômetro didático do professor; não é etapa de produção.

## 10) Conclusões executivas + Checklist FP&A

### Conclusões executivas
- Carteira média engana; o **mix de perfis** é o que direciona decisão.
- K-Means cria segmentos por semelhança e PCA torna isso visível para negócio.
- Tabela por cluster transforma técnica em governança (mix, risco, ticket, canal e LTV).
- Nomear clusters é o passo que conecta análise a plano de ação.
- Em produção, segmentação deve ser recalculada periodicamente e monitorada contra drift.

### Checklist FP&A (1 minuto)
- Features padronizadas antes do K-Means.
- K escolhido por interpretabilidade (silhouette como checagem opcional).
- PCA usado para comunicação executiva.
- Tabela por cluster pronta para decisão.
- Ações e dono por cluster definidos.
- Monitoramento do mix de carteira ao longo do tempo.